In [1]:
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import pickle
import scipy.stats
import itertools

In [2]:
from bci_plot.metadata import robot_sessions

In [3]:
'''
def get_total_movement_time(result):
    idxs = np.nonzero(result['movement_cum_dt'] == 0)[0]
    idxs = idxs[idxs > 0]
    out = result['movement_cum_dt'][idxs-1].sum()
    return out
'''
MAX_ACTION_MOVING_TIME = 40.0
MAX_MOVING_TIME = 80.0
def get_moving_time(logs_i, threshold=0.010):
    '''
    returns arrays of the cumulative moving time for each trial in a log file
    '''
    try:
        #out = logs_i['movement_cum_dt'][logs_i['trial_end_idx']-1]
        out = []
        for i, j in zip(logs_i['trial_start_idx'], logs_i['trial_end_idx']):
            tmp = logs_i['movement_cum_dt'][i:j]
            idxs = np.nonzero(tmp == 0)[0]
            idxs = idxs[idxs > 0]
            mt = tmp[idxs-1].sum()
            if len(idxs) == 0:
                mt += 2*MAX_ACTION_MOVING_TIME
            elif len(idxs) == 1:
                mt += 1*MAX_ACTION_MOVING_TIME
            out.append(mt)
        out = np.array(out)
    except:
        out = []
        for i, j in zip(logs_i['trial_start_idx'], logs_i['trial_end_idx']):
            moving_ticks = i + np.nonzero(logs_i['z_state'][i:j+1] == 0)[0]
            moving_ticks = moving_ticks[moving_ticks>=1]
            dts = (logs_i['timestamp'][moving_ticks] - logs_i['timestamp'][moving_ticks-1])
            dts = dts[dts < threshold]
            out.append(dts.sum())
        out = np.array(out)
        print('using timestamp')
    return out

In [4]:
sessions = list(robot_sessions.hover_ab_meta.keys())

In [5]:
logs = {}
meta = {}
for session in robot_sessions.hover_ab_meta:
    logs[session] = []
    for log_fname in robot_sessions.hover_ab_meta[session]:
        pickle_fname = log_fname.replace('.log', '.pickle')
        with open(f'../../data/robot/{session}/{pickle_fname}', 'rb') as f:
            logs[session].append(pickle.load(f))
    print(session)
    trial_length = np.hstack([item['timestamp'][item['trial_end_idx']] - item['timestamp'][item['trial_start_idx']] for item in logs[session]])
    trial_end_condition = np.hstack([item['trial_end_condition'] for item in logs[session]])
    trial_hover_state = np.hstack([item['hover_state'][item['trial_start_idx']] for item in logs[session]])
    n_successful_actions = np.hstack([[(item['gripper_retval'][i:j]==1).sum() for i,j in zip(item['trial_start_idx'], item['trial_end_idx'])] for item in logs[session]])
    n_unsuccessful_actions = np.hstack([[(item['gripper_retval'][i:j]==0).sum() for i,j in zip(item['trial_start_idx'], item['trial_end_idx'])] for item in logs[session]])
    n_actions = n_successful_actions + n_unsuccessful_actions
    moving_time = np.hstack([get_moving_time(item) for item in logs[session]])
    
    #((block, x), is_hover, (pick_status, place_status))
    # pick_status:  1: correct block
    #              -1: incorrect block
    #               0: no pick
    # place_status: 1: on correct x (subjective for hover)
    #              -1: not on correct x
    #               0: no place
    #              99: no pick
    outcomes = robot_sessions.outcomes_meta[session]
    (tmp0, hs, tmp1) = list(zip(*outcomes))
    (block_color, x_color) = list(zip(*tmp0))
    (pick_result, place_result) = list(zip(*tmp1))
    pick_result, place_result = np.array(pick_result), np.array(place_result)
    
    hover_successful_times = trial_length[(trial_hover_state==True )*(pick_result==1)*(place_result==1)]
    cv_successful_times    = trial_length[(trial_hover_state==False)*(pick_result==1)*(place_result==1)]
    hover_moving_times     = moving_time[(trial_hover_state==True )*(pick_result==1)*(place_result==1)]
    cv_moving_times        = moving_time[(trial_hover_state==False)*(pick_result==1)*(place_result==1)]
    hover_n_actions        = n_actions[(trial_hover_state==True )*(pick_result==1)*(place_result==1)]
    cv_n_actions           = n_actions[(trial_hover_state==False)*(pick_result==1)*(place_result==1)]
    hover_n_actions_all    = n_actions[(trial_hover_state==True )]
    cv_n_actions_all       = n_actions[(trial_hover_state==False)]
    hover_moving_times_all = moving_time[(trial_hover_state==True )]
    cv_moving_times_all    = moving_time[(trial_hover_state==False)]
    
    hover_success_percentage = ((place_result[trial_hover_state==True ]==1)*(pick_result[trial_hover_state==True ]==1)).mean()
    cv_success_percentage    = ((place_result[trial_hover_state==False]==1)*(pick_result[trial_hover_state==False]==1)).mean()
    
    meta[session] = {
        'trial_length': trial_length,
        'trial_end_condition': trial_end_condition,
        'trial_hover_state': trial_hover_state,
        'n_successful_actions': n_successful_actions,
        'n_unsuccessful_actions': n_unsuccessful_actions,
        'n_actions': n_actions,
        'moving_time': np.array(moving_time),
        'outcomes': outcomes,
        'block_color': block_color,
        'x_color': x_color,
        'pick_result': pick_result,
        'place_result': place_result,
        'hover_successful_times': hover_successful_times,
        'cv_successful_times': cv_successful_times,
        'hover_moving_times': hover_moving_times,
        'cv_moving_times': cv_moving_times,
        'hover_n_actions': hover_n_actions,
        'cv_n_actions': cv_n_actions,
        'hover_n_actions_all': hover_n_actions_all,
        'cv_n_actions_all': cv_n_actions_all,
        'hover_success_percentage': hover_success_percentage,
        'cv_success_percentage': cv_success_percentage,
        'hover_moving_times_all': hover_moving_times_all,
        'cv_moving_times_all': cv_moving_times_all,
    }
meta['exp_2024-06-06_12-37-30']['subject'] = 'H2'
meta['exp_2024-06-10_14-41-45']['subject'] = 'H1'
meta['exp_2024-06-10_15-53-08']['subject'] = 'H1'
meta['exp_2024-06-13_15-08-39']['subject'] = 'H4'
meta['exp_2024-06-13_16-05-13']['subject'] = 'H4'
meta['exp_2024-06-26_12-45-37']['subject'] = 'S2'
meta['exp_2024-07-09_15-01-12']['subject'] = 'S2'

stats = {}
stats['H2'] = {k: meta[k] for k in ['exp_2024-06-06_12-37-30']}
stats['H1'] = {k: meta[k] for k in ['exp_2024-06-10_14-41-45', 'exp_2024-06-10_15-53-08']}
stats['H4'] = {k: meta[k] for k in ['exp_2024-06-13_15-08-39', 'exp_2024-06-13_16-05-13']}
stats['S2'] = {k: meta[k] for k in ['exp_2024-06-26_12-45-37', 'exp_2024-07-09_15-01-12']}

with open('../../data/robot_compiled/robot_compiled_stats.pickle', 'wb') as f:
    pickle.dump(stats, f)

exp_2024-06-06_12-37-30
exp_2024-06-10_14-41-45
exp_2024-06-10_15-53-08
exp_2024-06-13_15-08-39
exp_2024-06-13_16-05-13
exp_2024-06-26_12-45-37
exp_2024-07-09_15-01-12
